# Build a Baseline with Langchain

Having prepared a test dataset, we are now equipped to conduct experiments and iteratively enhance our RAG pipelines.

## Introducing Metrics Driven Development (MDD)

A key challenge is developing a systematic approach to measure and refine our RAG pipeline. To address this, we propose `Metrics Driven Development` (MDD), inspired by the popular Test Driven Development methodology. MDD advocates for employing various metrics to assess different facets of the LLM application and conducting targeted experiments for improvement based on specific use cases.

Metrics Driven Development (MDD) offers a structured framework to tackle the complexities involved in optimizing RAG applications. Below is a mind map created by an engineer at AWS, illustrating potential strategies for enhancing a RAG pipeline.

![](https://media.licdn.com/dms/image/D4D22AQEgjWxKXokOPA/feedshare-shrink_800/0/1708498751086?e=1711584000&v=beta&t=xaT95vKS8m4qTybofpKqQfXOGoFs8lQXBuOk2Fr45AE)

The [original Miro mind map](https://miro.com/app/board/uXjVNvklNmc=/) is accessible for further exploration. MDD serves as a guiding light through the complexity of enhancing RAG applications.

Let's proceed to see MDD in action.

In [ ]:
%%capture --no-stderr
%pip install -U "unstructured[md]" chromadb langchain_openai langchainhub

In [ ]:
import os
import getpass

os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")

## Building a baseline

if you remember, in the [last notebook](./dataset.ipynb) we outline the steps and addressed a few

1. Load the data as documents. ✅
2. Generate the test set from these documents. ✅
3. Upload and verify the test set with Langsmith. ✅
4. Formulate experiments to improve you RAG pipeline. ⏳
5. Choose the right metrics to evaluate the experiment ⏳
6. Analyze the results using the Langsmith dashboard. ⏳

So lets continue where we left off.

## 4. Formulate experiments to improve your RAG pipeline

Now for the baseline first thing we want to know is how effective is vanila GPT-3.5 compared to RAG based model. RAG should be superiour because there is specific information about companies but what exactly is the difference?

> is RAG better than just using an LLM for our case?

In order to compare that need to create 2 chains 
1. Just LLM - gpt-3.5
2. LLM + Retriver 

### Building the RAG

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

To build the RAG lets load the data, chunk it and add it to a vector store for retrieval. If you want more info on how to build RAG systems with langchain, check the [docs](https://python.langchain.com/docs/modules/data_connection/)

In [2]:
# load the documents
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader("./data/")
documents = loader.load()

# add filename as metadata
for document in documents:
    document.metadata["file_name"] = document.metadata["source"]

# how many docs do we have?
docs = documents
len(docs)

26

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# create the vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [4]:
# get one example question for the dataset for testing
from langsmith import Client

client = Client()
examples = list(client.list_examples(dataset_name="basecamp"))

q = examples[0].inputs
q

{'question': "What is the importance of levelheadedness in the company's values?"}

In [5]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain import hub

In [6]:
# Retrieve and generate using the relevant snippets from the docs
vectorstore_retriever = vectorstore.as_retriever()
# load a RAG prompt from Langchain HUB
prompt = hub.pull("rlm/rag-prompt")
# our llm of choice
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def ragas_output_parser(docs):
    return [doc.page_content for doc in docs]

Now lets string together all the components together and make the RAG pipeline

In [7]:
from langchain_core.runnables import RunnableParallel

generator = prompt | llm | StrOutputParser()

retriever = RunnableParallel(
    {
        "context": vectorstore_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
)

filter_langsmith_dataset = RunnableLambda(
    lambda x: x["question"] if isinstance(x, dict) else x
)

rag_chain = RunnableParallel(
    {
        "question": filter_langsmith_dataset,
        "answer": filter_langsmith_dataset | retriever | generator,
        "contexts": filter_langsmith_dataset
        | vectorstore_retriever
        | ragas_output_parser,
    }
)

In [8]:
# check with the example question to see if everything is working
get_answer = RunnableLambda(lambda x: x["answer"])
resp = (rag_chain | get_answer).invoke(q)
resp

"Levelheadedness is important in the company's values because it promotes calm, considered, and thoughtful interactions with others. It helps prevent acting out of spite, rushing to judgment, or jumping to conclusions. This approach allows for clear communication and effective problem-solving within the company."

Voilà! We have our RAG working with Langchain. Go on and try a few questions yourself from the `examples` we generated.

Now, let's build the LLM.

### Just the LLM

Setting this up is much easier as you could imagine, all you need are the prompts.

In [9]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

Question: {question}

Helpful Answer:"""
llm_prompt = PromptTemplate.from_template(template)

just_llm = (
    {"question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
    | RunnableParallel(
        {
            "answer": RunnablePassthrough(),
            "contexts": RunnableLambda(lambda _: [""]),
        }
    )
)

In [10]:
resp = (just_llm | get_answer).invoke(q)
resp

"Levelheadedness is important in the company's values because it promotes rational decision-making, reduces conflict, and fosters a positive work environment. It helps employees stay calm under pressure and think critically about challenges they may face. Thanks for asking!"

Try out a few `examples` from this chain also, see if you can spot any differences in performance by eyeballing the results.

## 5. Choose the right metrics to evaluate the experiment

Ragas provides you with a different metrics that you can use to measure the different components of your RAG pipeline. You can see the entire list in the [docs](https://docs.ragas.io/en/latest/concepts/metrics/index.html).

For this experiment we are going to choose [Answer Correctness](https://docs.ragas.io/en/latest/concepts/metrics/answer_correctness.html). `Answer Correctness` is an end-to-end metric that measures the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness. Do check out the docs to learn more about how it works internally.

To make evaluation of Langchain chains on Langsmith easier, Ragas provides you with 2 utils 
1. `EvaluatorChain`: which is a langchain chain that take a Ragas metric and creates a `Chain` which outputs the score.
2. `evaluate()`: this is a util function for Langsmith that takes a dataset_name, chain and metrics to run the evaluations.

Lets take a look at both of them.

### `EvaluatorChain`

Lets create one for `Answer Correctness` and evaluate both of the baselines we created

In [11]:
from ragas.integrations.langchain import EvaluatorChain

# the metric we will be using
from ragas.metrics import answer_correctness

### `evaluate()` Langsmith Dataset

this utility function take the Langsmith dataset_name, RAG chain, the Ragas metrics you choose and runs the evaluations for you.

In [12]:
from ragas.integrations.langsmith import evaluate

Lets evaluate the `rag_chain` first.

In [13]:
dataset_name = "basecamp"
# evaluate just llms
run = evaluate(
    dataset_name=dataset_name,
    llm_or_chain_factory=rag_chain,
    experiment_name="rag_chain_1",
    metrics=[answer_correctness],
    verbose=True,
)

View the evaluation results for project 'rag_chain_1' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/fe31443e-60c7-462f-87e7-35a85894055f/compare?selectedSessions=92cc8bb4-5b83-4033-ba6c-f5ddbfedab93

View all tests for Dataset basecamp at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/fe31443e-60c7-462f-87e7-35a85894055f
[>                                                 ] 0/21

/Users/wfh/code/lc/community/ragas-notes/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[------------------------------------------------->] 21/21

,feedback.answer_correctness,error,execution_time,run_id
count,21.000000,0,21.000000,21
unique,NaN,0,NaN,21
top,NaN,NaN,NaN,4ee9464b-8da9-40a1-ae64-528c2d7f04f2
freq,NaN,NaN,NaN,1
mean,0.528093,NaN,2.032485,NaN
std,0.177913,NaN,1.014589,NaN
min,0.181416,NaN,0.601767,NaN
25%,0.405035,NaN,1.511995,NaN
50%,0.562180,NaN,1.725981,NaN
75%,0.667870,NaN,2.092583,NaN


Now lets evaluate the RAG pipeline

In [ ]:
# evaluate rag_chain
run = evaluate(
    dataset_name=dataset_name,
    llm_or_chain_factory=just_llm,
    experiment_name="just_llm_1",
    metrics=[answer_correctness],
    verbose=True,
)

View the evaluation results for project 'just_llm_1' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/fe31443e-60c7-462f-87e7-35a85894055f/compare?selectedSessions=5e1f0015-3911-403f-9ef2-f76ed83ff3e6

View all tests for Dataset basecamp at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/fe31443e-60c7-462f-87e7-35a85894055f
[----------------------------------------------->  ] 20/21

Now you can check you langsmith dataset dashboard to view and analyise the results.